In [1]:
import sys
import os
import nest_asyncio
import asyncpg
from pyspark.sql import SparkSession
import pandas as pd
import xml.etree.ElementTree as et


In [2]:

nest_asyncio.apply()

DB_CONFIG = {
    "user": os.getenv("POSTGRES_USER", "admin"),
    "password": os.getenv("POSTGRES_PASSWORD", "admin"),
    "database": os.getenv("POSTGRES_DB", "tender"),
    "host": os.getenv("POSTGRES_HOST", "postgre"),
    "port": int(os.getenv("POSTGRES_PORT", 5432))
}

async def read_table_as_df() -> pd.DataFrame:
    conn = await asyncpg.connect(**DB_CONFIG)
    rows = await conn.fetch("""
        SELECT * FROM xml_storage WHERE avro_path IS NULL
    """)
    df = pd.DataFrame([dict(row) for row in rows])
    await conn.close()
    return df

pandas_df = await read_table_as_df()

id_list = pandas_df['id'].tolist()

if pandas_df.empty:
    raise SystemExit("Dataframe is empty, exiting program.")

In [3]:
print(pandas_df)

   id                                           xml_data avro_path
0   1  <awards_huf><award><date>2021-01-17</date><dea...      None


In [4]:
from dataclasses import dataclass
from datetime import date
from decimal import Decimal

@dataclass
class Award:
    date: date
    deadline_date: date
    title: str
    category: str
    description: str
    phase: str
    place: str
    awarded_value: Decimal
    awarded_currency: str
    awarded_date: date
    suppliers_name: str
    count: int
    offers_count: int

@dataclass
class AwardsHuf:
    awards: list[Award]


In [5]:
pandas_df = pandas_df.where(pandas_df.notnull())

pandas_df = pandas_df.drop(['id', 'avro_path'], axis=1)

print(pandas_df)

                                            xml_data
0  <awards_huf><award><date>2021-01-17</date><dea...


In [6]:
xml_strings = pandas_df['xml_data'].tolist()

def convert_xml_to_dict(xml_str):
    tree = et.ElementTree(et.fromstring(xml_str))
    root = tree.getroot()

    data = {}
    for i, child in enumerate(root):
        data[i] = []
        for ch in child:
            data[i].append(ch.text)
    return data

xml_dicts = [convert_xml_to_dict(xml) for xml in xml_strings]


# Convert the dictionary to dataclass instances
def convert_dict_to_award(xml_dict):
    awards = []
    for i, values in xml_dict.items():
        award = Award(
            date=date.fromisoformat(values[0]),
            deadline_date=date.fromisoformat(values[1]),
            title=values[2],
            category=values[3],
            description=values[4],
            phase=values[5],
            place=values[6],
            awarded_value=Decimal(values[7]),
            awarded_currency=values[8],
            awarded_date=date.fromisoformat(values[9]),
            suppliers_name=values[10],
            count=int(values[11]),
            offers_count=int(values[12])
        )
        awards.append(award)
    return AwardsHuf(awards)

awards_huf_list = [convert_dict_to_award(xml_dict) for xml_dict in xml_dicts]

print(awards_huf_list[0].awards[0])

Award(date=datetime.date(2021, 1, 17), deadline_date=datetime.date(2021, 1, 28), title='Bölcsőde építése Dömösön – TOP-1.4.1-19-KO1-2019', category='constructions', description='Dömösi Szivárvány Óvoda bővítése 8 férőhelyes mini bölcsödével. Rendeltetés: óvoda, bölcsőde (középület) Építési övezet: Lf-9 (falusias beépítés) Épületelhelyezés: oldalhatáron álló Meglévő épület bruttó alapterülete: 242,4 m2 Tervezett épületrész bruttó alapterülete: 92,8 m2 Teljes épület bruttó alapterülete: 335,2 m2 A beépítettséghez szükséges bruttó alapterület: 335,2 m2 Meglévő épület nettó alapterülete: 185,3 m2 Tervezett bővítés nettó alapterülete: 76,2m2 Épület nettó alapterülete összesen: 261,5 m2 A részletes műszaki leírást a közbeszerzési dokumentáció tartalmazza. Ajánlatkérő felhívja a figyelmet a 321/2015. (X.30) Korm. rendelet 46. § (3) bekezdésében foglaltakra.', phase='E60 - Szerződéskötési-, teljesítési szakasz', place='HU212 Komárom-Esztergom', awarded_value=Decimal('51809732.0'), awarded_curr

In [7]:
import random

for awards in awards_huf_list:
    for idx, award in enumerate(awards.awards):
        if idx % 2 == 0:
            description = None
        if idx % 3 == 0:
            awarded_value = -1500000
        if idx % 4 == 0:
            awarded_currency = random.choice(["HUF", "EUR", "INVALID_DATA"])
        if idx % 5 == 0:
            place = None

print(awards)

AwardsHuf(awards=[Award(date=datetime.date(2021, 1, 17), deadline_date=datetime.date(2021, 1, 28), title='Bölcsőde építése Dömösön – TOP-1.4.1-19-KO1-2019', category='constructions', description='Dömösi Szivárvány Óvoda bővítése 8 férőhelyes mini bölcsödével. Rendeltetés: óvoda, bölcsőde (középület) Építési övezet: Lf-9 (falusias beépítés) Épületelhelyezés: oldalhatáron álló Meglévő épület bruttó alapterülete: 242,4 m2 Tervezett épületrész bruttó alapterülete: 92,8 m2 Teljes épület bruttó alapterülete: 335,2 m2 A beépítettséghez szükséges bruttó alapterület: 335,2 m2 Meglévő épület nettó alapterülete: 185,3 m2 Tervezett bővítés nettó alapterülete: 76,2m2 Épület nettó alapterülete összesen: 261,5 m2 A részletes műszaki leírást a közbeszerzési dokumentáció tartalmazza. Ajánlatkérő felhívja a figyelmet a 321/2015. (X.30) Korm. rendelet 46. § (3) bekezdésében foglaltakra.', phase='E60 - Szerződéskötési-, teljesítési szakasz', place='HU212 Komárom-Esztergom', awarded_value=Decimal('51809732

In [8]:
spark = SparkSession.builder \
                    .appName("xml_to_avro") \
                    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.1") \
                    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/lib/python3.12/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3c8f9424-25e0-46b9-865e-10f9e9dc7416;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.5.1 in central
	found org.tukaani#xz;1.9 in central
:: resolution report :: resolve 181ms :: artifacts dl 4ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.5.1 from central in [default]
	org.tukaani#xz;1.9 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------------------
:: retrievin

In [9]:
from pyspark.sql import Row

rows = []

for awards in awards_huf_list:
    for award in awards.awards:
        row = Row(
            date=award.date,
            deadline_date=award.deadline_date,
            title=award.title,
            category=award.category,
            description=award.description,
            phase=award.phase,
            place=award.place,
            awarded_value=str(award.awarded_value),  # PySpark Decimal handling can vary
            awarded_currency=award.awarded_currency,
            awarded_date=award.awarded_date,
            suppliers_name=award.suppliers_name,
            count=award.count,
            offers_count=award.offers_count
        )
        rows.append(row)

df = spark.createDataFrame(rows)

In [10]:
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- deadline_date: date (nullable = true)
 |-- title: string (nullable = true)
 |-- category: string (nullable = true)
 |-- description: string (nullable = true)
 |-- phase: string (nullable = true)
 |-- place: string (nullable = true)
 |-- awarded_value: string (nullable = true)
 |-- awarded_currency: string (nullable = true)
 |-- awarded_date: date (nullable = true)
 |-- suppliers_name: string (nullable = true)
 |-- count: long (nullable = true)
 |-- offers_count: long (nullable = true)



In [11]:
from pyspark.sql import functions as F

# Fix wrong xml

# Step 1: Replace None in 'description' with "We could not find description"
df = df.withColumn(
    'description',
    F.when(F.col('description').isNull(), "We could not find description").otherwise(F.col('description'))
)

# Step 2: Filter out rows where 'awarded_value' is negative
df = df.filter(F.col('awarded_value') >= 0)

# Step 3: Filter rows where 'awarded_currency' is not 'HUF'
df = df.filter(F.col('awarded_currency') == 'HUF')

# Step 4: Replace None in 'place' with "Unknown place"
df = df.withColumn(
    'place',
    F.when(F.col('place').isNull(), "Unknown place").otherwise(F.col('place'))
)

df.show()


+----------+-------------+--------------------+-------------+--------------------+--------------------+--------------------+-------------+----------------+------------+--------------------+-----+------------+
|      date|deadline_date|               title|     category|         description|               phase|               place|awarded_value|awarded_currency|awarded_date|      suppliers_name|count|offers_count|
+----------+-------------+--------------------+-------------+--------------------+--------------------+--------------------+-------------+----------------+------------+--------------------+-----+------------+
|2021-01-17|   2021-01-28|Bölcsőde építése ...|constructions|Dömösi Szivárvány...|E60 - Szerződéskö...|HU212 Komárom-Esz...|   51809732.0|             HUF|  2021-11-17|Mészáros Épületgé...|    1|           3|
|2021-01-28|   2021-03-03|Építési beruházás...|constructions|Ajánlatkérő az EF...|E60 - Szerződéskö...|         HU332 Békés|  204326256.0|             HUF|  2021-11

In [12]:
df.count()

44

In [19]:
from pathlib import Path

output_path = Path.cwd() / "awards_data.avro"

df.write.format("avro").mode("overwrite").save(str(output_path))

In [21]:
import asyncpg

async def insert_avro_path(id_list, output_path) -> None:
    conn = await asyncpg.connect(**DB_CONFIG)

    output_path_str = str(output_path)
    
    for i in id_list:
        await conn.execute("""
            UPDATE xml_storage
            SET avro_path = $1
            WHERE id = $2
        """, output_path_str, i)

    await conn.close()

await insert_avro_path(id_list, output_path)


In [24]:
print(f"Avro path ({id_list}) inserted for:{output_path} ")

print("END")

Avro path ([1]) inserted for:/opt/workspace/awards_data.avro 
END
